# Plot paired and unpaired synapses

In [ ]:
# Load up all of the libraries we need

from deriva.core import HatracStore, ErmrestCatalog, get_credential
from synspy.analyze.pair import SynapticPairStudy

import synapse_utils

import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
credential = get_credential("synapse.isrd.isi.edu")
ermrest_catalog = ErmrestCatalog('https', 'synapse.isrd.isi.edu', 1, credential)
hatrac_store = HatracStore('https', 'synapse.isrd.isi.edu', credentials=credential)

Lets start by getting a list of all of the studies currently in the server. We will end up with a list of studies, where each study has been classified by the study type based on the protocol that was followed. 

In [ ]:
# Get the current list of studies from the server.
study_entities = synapse_utils.get_synapse_studies()
study_uri = study_entities.uri

print('Identified %d studies' % len(study_entities))

In [ ]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',
    'PrcDsy20170613A': 'conditioned',
    'PrcDsy20170615A': 'unconditioned',
    'PrcDsy20170613B': 'fullcycle-control',
    'PrcDsy20171030A': 'groundtruth-control',
    'PrcDsy20171030B': 'interval-groundtruth-control'
}

for i in study_entities:
    if protocol_types[i['Protocol']] == 'aversion':
        i['Type'] = 'learner' if i['Learner'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['Protocol']]
    study_list.append(i)

Lets make some convienent subsets of studies

In [ ]:
learners = [i for i in study_list if i['Type'] == 'learner']
nonlearners = [i for i in study_list if i['Type'] == 'nonlearner']

print([i['Study'] for i in learners if True])

Pick a study that we want to look at and set the parameters for the sweep.

In [ ]:
syn_study_id = study_list[49]['Study'] # SynStd8175, the study we'll analyze
syn_pair_radii = (2.0, 3.0, 4.0) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

Now calculate the distances and save the results into dataframes....

In [ ]:
study = SynapticPairStudy.from_study_id(ermrest_catalog, syn_study_id)
study.retrieve_data(hatrac_store)

s1_to_s2, s2_to_s1 = study.syn_pairing_maps(syn_pair_radii, syn_dx_core_ratio, syn_core_max_ratio)

# get results for different radii, store them in a dictonary of pandas

unpaired_before, unpaired_after, paired_before, paired_after = {}, {}, {}, {}

# Create a list of pandas, one for each radii
for i,r in enumerate(syn_pair_radii):
    unpaired1 = study.get_unpaired(s1_to_s2[i,:], study.s1)
    unpaired2 = study.get_unpaired(s2_to_s1[i,:], study.s2)
    paired1, paired2 = study.get_pairs(s1_to_s2[i,:], study.s1, study.s2)
    
    unpaired_before[r] = pd.DataFrame(unpaired1[:,0:5], columns=['x','y','z','core','hollow'])
    unpaired_after[r] = pd.DataFrame(unpaired2[:,0:5], columns=['x','y','z','core','hollow'])
    paired_before[r]  = pd.DataFrame(paired1[:,0:5], columns=['x','y','z','core','hollow'])
    paired_after[r] = pd.DataFrame(paired2[:,0:5], columns=['x','y','z','core','hollow'])

Compute some basic Statistics

In [ ]:
for r in  syn_pair_radii:
    print('Radius {0}'.format(r))
    print('    unpaired before {0}'.format(len(unpaired_before[r].index)))
    print('    unpaired after {0}'.format(len(unpaired_after[r].index)))
    print('    paired {0}'.format(len(paired_before[r].index)))

Plot histograms of different radii

In [ ]:
data = []

r = max(syn_pair_radii)
before = paired_before[r]
after = paired_after[r]
pair_distance = np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)
                
hist_data = go.Histogram(x=pair_distance, name='Radias {0}'.format(r))

hist_layout = go.Layout(
    title='Pair Histogram Radius {0}'.format(r),
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data= [hist_data], layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

In [ ]:
x = [1,2,3,4,5, 6, 7, 8,9,10,11,12]
i = 4
x[i*4:i*4+4] = [True] * 4
x

In [ ]:
data = []

for r in  syn_pair_radii:
    
    before = paired_before[r]
    after = paired_after[r]
    up_before = unpaired_before[r]
    up_after = unpaired_after[r]
  
    before_synapses = go.Scatter3d(
        visible = False,
        x=before['x'], y=before['y'], z=before['z'],
        legendgroup='paired',
        name='Before Paired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    after_synapses = go.Scatter3d(
        visible = False,
        x=after['x'], y=after['y'], z=after['z'],
        legendgroup='paired',
        name='After Paired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.5
        )
    )
    
    up_before_synapses = go.Scatter3d(
        visible = False,      
        x=up_before['x'], y=up_before['y'], z=up_before['z'],
        legendgroup='unpaired',
        name='Before Unpaired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    up_after_synapses = go.Scatter3d(
        visible = False,
        x=up_after['x'], y=up_after['y'], z=up_after['z'],
        legendgroup='unpaired',
        name='After Unpaired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

    data.extend([ before_synapses, after_synapses, up_before_synapses, up_after_synapses])

for i in range(4):
    data[i]['visible'] = True

steps = []
for i in range(len(data)//4):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
        label='Radius {0}'.format(syn_pair_radii[i])
    )
    x[i*4:i+3] = [False]*4
    step['args'][1][i*4:i+4] = [True] * 4 # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {"visible": False},
    pad = {"t": 50},
    steps = steps
)]

layout = go.Layout(
        height=800,
        title='Paired and Unpaired Synapapses',
        sliders=sliders
    )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='Sine Wave Slider')

In [ ]:
for r in  syn_pair_radii:
    
    before = paired_before[r]
    after = paired_after[r]
    up_before = unpaired_before[r]
    up_after = unpaired_after[r]
  
    before_synapses = go.Scatter3d(
        x=before['x'], y=before['y'], z=before['z'],
        legendgroup='paired',
        name='Before Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    after_synapses = go.Scatter3d(
        x=after['x'], y=after['y'], z=after['z'],
        legendgroup='paired',
        name='After Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.5
        )
    )
    
    up_before_synapses = go.Scatter3d(
        x=up_before['x'], y=up_before['y'], z=up_before['z'],
        legendgroup='unpaired',
        name='Before Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    up_after_synapses = go.Scatter3d(
        x=up_after['x'], y=up_after['y'], z=up_after['z'],
        legendgroup='unpaired',
        name='After Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

    data = [ before_synapses, after_synapses, up_before_synapses, up_after_synapses]

    layout = go.Layout(
        height=800,
        title='Paired Synapapses at Radius {0}'.format(r),
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)